In [1]:
import pandas as pd
import numpy as np
import json
import requests
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import time
from bs4 import BeautifulSoup
from bs4 import BeautifulStoneSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import tempfile
import time
from bs4 import BeautifulSoup

In [4]:
temp_df = pd.DataFrame()

for j in range(1,50):
    CHROMEDRIVER_PATH = r"C:\Users\moham\chrome_driver\chromedriver-win64\chromedriver.exe"
    
    # Temp profile for fresh session
    temp_profile = tempfile.mkdtemp()
    
    # Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument(f"--user-data-dir={temp_profile}")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")
    
    # Create service and driver
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Load website
    start = time.time()
    driver.get("https://www.ambitionbox.com/list-of-companies?page={}".format(j))
    time.sleep(5)
    html = driver.page_source
    driver.quit()
    end = time.time()
    
    # Parse page
    soup = BeautifulSoup(html, "html.parser")
    print(f"Request took {end - start:.2f} seconds")
    print("Page Title:", soup.title.text.strip())
    # print(soup.prettify())
    com_list=[]
    for i in (soup.find_all('h2')):
        # print(i.text.strip())
        com_list.append(i.text.strip())
    
    company_series_data=pd.Series(com_list[:20])    
    
        # to find the company type likeits types in which it is located and what type is it 
    company_type_list=[]
    companies_types = soup.find_all('span', class_='companyCardWrapper__interLinking')
    
    for company_type in companies_types:
        # print(company_type.text.strip().replace('|','-').replace('/',','))
        company_type_list.append(company_type.text.strip().replace('|','-').replace('/',','))
    company_type_series=pd.Series(company_type_list)
        # to find the rating and then converting that into the list for the dataframe 
    ratings = soup.find_all('div', style=lambda x: x and 'padding-bottom' in x)
    rating_list=[]
    for r in ratings:
        rating_text = r.text.strip()
        if rating_text.replace('.', '', 1).isdigit():  # crude check for float-like values
            #print(rating_text)
            rating_list.append((rating_text+"k"))
    # print(rating_list)
    rating_series=pd.Series(rating_list)
        # Get all company cards
    company_cards = soup.find_all('div', class_='companyCardWrapper')
    
    review_list = []
    
    for card in company_cards:
        action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
        if len(action_counts) > 1:
            review = action_counts[0].text.strip()
            review_list.append(review)
    
    # Convert to Series
    review_series = pd.Series(review_list)
    # print(review_series)

    
        # Get all company cards
    company_cards = soup.find_all('div', class_='companyCardWrapper')
    
    salary_list = []
    
    for card in company_cards:
        action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
        if len(action_counts) > 0:
            salary = action_counts[1].text.strip()
            salary_list.append(salary)
    
    # Convert to Series
    salary_series = pd.Series(salary_list)
    # print(salary_series)
    
        # Get all company cards
    company_cards = soup.find_all('div', class_='companyCardWrapper')
    
    job_list = []
    
    for card in company_cards:
        action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
        if len(action_counts) > 3:
            job = action_counts[3].text.strip()
            job_list.append(job)
    
    # Convert to Series
    job_series = pd.Series(job_list)
    # print(job_series)
    
        # Get all company cards
    company_cards = soup.find_all('div', class_='companyCardWrapper')
    
    interview_list = []
    
    for card in company_cards:
        action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
        if len(action_counts) > 2:
            interview = action_counts[2].text.strip()
            interview_list.append(interview)
    
    # Convert to Series
    interview_series = pd.Series(interview_list)
    # print(interview_series)

        # Get all company cards
    company_cards = soup.find_all('div', class_='companyCardWrapper')
    
    highly_rated_list = []
    
    for card in company_cards:
        # Check if the "Highly Rated For" label is present
        high_label = card.find('span', class_='companyCardWrapper__ratingHeader--high')
        
        if high_label and 'Highly Rated For' in high_label.text:
            # Extract the values next to it
            high_values = card.find('span', class_='companyCardWrapper__ratingValues')
            if high_values:
                highly_rated_list.append(high_values.text.strip())
            else:
                highly_rated_list.append('N/A')
        else:
            highly_rated_list.append('N/A')
    
    # Convert to Series
    highly_rated_series = pd.Series(highly_rated_list)
    # print(highly_rated_series)

        # Extracting "Critically Rated For" values
    critical_list = []
    
    for card in company_cards:
        # Check if the critical rating section exists (corrected class)
        low_label = card.find('span', class_='companyCardWrapper__ratingHeader--critical')
        
        if low_label and 'Critically Rated For' in low_label.text:
            # Find the associated rating values (next span)
            low_values = low_label.find_next('span', class_='companyCardWrapper__ratingValues')
            if low_values:
                critical_list.append(low_values.text.strip())
            else:
                critical_list.append('N/A')
        else:
            critical_list.append('N/A')
    
    # Convert to Series
    critical_series = pd.Series(critical_list)
    # print(critical_series)
    
    
    # Combine into DataFrame
    df = pd.DataFrame({
        'Company': company_series_data,
        'Rating': rating_series,
        "Reviews":review_series,
        "company_type":company_type_series,
        'salary':salary_series,
        "Interview_count": interview_series,
        "Jobs_available":job_series,
        "highly_rated_for":highly_rated_series,
        "critically_rated_for":critical_series
    })
    print(df)
    temp_df = pd.concat([temp_df, df], ignore_index=True)


temp_df.to_csv("ambition_data_1000.csv")
    
    


Request took 11.42 seconds
Page Title: Companies in India | AmbitionBox
                   Company Rating Reviews  \
0                      TCS   3.6k   97.1k   
1                Accenture   3.8k   62.6k   
2                    Wipro   3.7k   56.8k   
3                Cognizant   3.7k     54k   
4                Capgemini   3.7k   45.6k   
5                HDFC Bank   3.9k     43k   
6                  Infosys   3.6k   42.5k   
7               ICICI Bank   4.0k   41.6k   
8                  HCLTech   3.5k   39.2k   
9            Tech Mahindra   3.5k   37.8k   
10                 Genpact   3.8k   35.4k   
11         Teleperformance   3.9k   32.1k   
12  Concentrix Corporation   3.7k   28.3k   
13               Axis Bank   3.7k   27.9k   
14                  Amazon   4.0k     27k   
15                     Jio   4.0k   25.3k   
16              iEnergizer   4.7k   24.4k   
17         Reliance Retail   3.9k     24k   
18                     IBM   4.0k   23.4k   
19             LTIMindtree  

In [ ]:
CHROMEDRIVER_PATH = r"C:\Users\moham\Downloads\chromedriver-win64\chromedriver.exe"


In [ ]:
# url = "https://www.ambitionbox.com/list-of-companies"

# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
# }

# response = requests.get(url, headers=headers)

In [ ]:
requests.get("https://www.ambitionbox.com/list-of-companies?page=1").text

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import tempfile
import time
from bs4 import BeautifulSoup

# Path to chromedriver.exe
CHROMEDRIVER_PATH = r"C:\Users\moham\chrome_driver\chromedriver-win64\chromedriver.exe"

# Temp profile for fresh session
temp_profile = tempfile.mkdtemp()

# Chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument(f"--user-data-dir={temp_profile}")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

# Create service and driver
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load website
start = time.time()
driver.get("https://www.ambitionbox.com/list-of-companies?page=1")
time.sleep(5)
html = driver.page_source
driver.quit()
end = time.time()

# Parse page
soup = BeautifulSoup(html, "html.parser")
print(f"Request took {end - start:.2f} seconds")
print("Page Title:", soup.title.text.strip())


In [ ]:
soup.find_all('h1')[0].text.strip()

In [ ]:
com_list=[]
for i in (soup.find_all('h2')):
    print(i.text.strip())
    com_list.append(i.text.strip())

company_series_data=pd.Series(com_list[:20])    
company_series_data

In [ ]:
# to find the company type likeits types in which it is located and what type is it 
company_type_list=[]
companies_types = soup.find_all('span', class_='companyCardWrapper__interLinking')

for company_type in companies_types:
    print(company_type.text.strip().replace('|','-').replace('/',','))
    company_type_list.append(company_type.text.strip().replace('|','-').replace('/',','))
company_type_series=pd.Series(company_type_list)

In [ ]:
# to find the rating and then converting that into the list for the dataframe 
ratings = soup.find_all('div', style=lambda x: x and 'padding-bottom' in x)
rating_list=[]
for r in ratings:
    rating_text = r.text.strip()
    if rating_text.replace('.', '', 1).isdigit():  # crude check for float-like values
        #print(rating_text)
        rating_list.append((rating_text+"k"))
print(rating_list)

In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

review_list = []

for card in company_cards:
    action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
    if len(action_counts) > 1:
        review = action_counts[0].text.strip()
        review_list.append(review)

# Convert to Series
review_series = pd.Series(review_list)
print(review_series)


In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

salary_list = []

for card in company_cards:
    action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
    if len(action_counts) > 0:
        salary = action_counts[1].text.strip()
        salary_list.append(salary)

# Convert to Series
salary_series = pd.Series(salary_list)
print(salary_series)


In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

job_list = []

for card in company_cards:
    action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
    if len(action_counts) > 3:
        job = action_counts[3].text.strip()
        job_list.append(job)

# Convert to Series
job_series = pd.Series(job_list)
print(job_series)


In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

interview_list = []

for card in company_cards:
    action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
    if len(action_counts) > 2:
        interview = action_counts[2].text.strip()
        interview_list.append(interview)

# Convert to Series
interview_series = pd.Series(interview_list)
print(interview_series)


In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

highly_rated_list = []

for card in company_cards:
    # Check if the "Highly Rated For" label is present
    high_label = card.find('span', class_='companyCardWrapper__ratingHeader--high')
    
    if high_label and 'Highly Rated For' in high_label.text:
        # Extract the values next to it
        high_values = card.find('span', class_='companyCardWrapper__ratingValues')
        if high_values:
            highly_rated_list.append(high_values.text.strip())
        else:
            highly_rated_list.append('N/A')
    else:
        highly_rated_list.append('N/A')

# Convert to Series
highly_rated_series = pd.Series(highly_rated_list)
print(highly_rated_series)


In [ ]:
# Extracting "Critically Rated For" values
critical_list = []

for card in company_cards:
    # Check if the critical rating section exists (corrected class)
    low_label = card.find('span', class_='companyCardWrapper__ratingHeader--critical')
    
    if low_label and 'Critically Rated For' in low_label.text:
        # Find the associated rating values (next span)
        low_values = low_label.find_next('span', class_='companyCardWrapper__ratingValues')
        if low_values:
            critical_list.append(low_values.text.strip())
        else:
            critical_list.append('N/A')
    else:
        critical_list.append('N/A')

# Convert to Series
critical_series = pd.Series(critical_list)
print(critical_series)


In [ ]:
series_data = pd.Series(com_list[:20])
rating_data = pd.Series(rating_list[:20])

# Combine into DataFrame
df = pd.DataFrame({
    'Company': series_data,
    'Rating': rating_data,
    "Reviews":review_series,
    "company_type":company_type_series,
    'salary':salary_series,
    "Interview_count":interviews_series,
    "Jobs_available":job_series,
    "highly_rated_for":highly_rated_series,
    "critically_rated_for":critical_series
})

print(df)
df.to_csv('Ambition_box_data.csv')

In [ ]:
# this code is for fetching the reviews from the website and then converting them into the list for the dataframe 
import re

reviews_list=[]
rating_counts = soup.find_all('span', class_='companyCardWrapper__companyRatingCount')

for rating in rating_counts:
    print(rating.text.strip())
    reviews_list.append(rating.text.strip())

cleaned_reviews=[r.strip('()') for r in reviews_list]

review_series=pd.Series(cleaned_reviews)



In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

salaries_list = []

for card in company_cards:
    action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
    if action_counts:
        salary = action_counts[1].text.strip()  # First item is Salary
        salaries_list.append(salary)

# Convert to Series
salary_series = pd.Series(salaries_list)
print(salary_series)


In [ ]:
# Get all company cards
company_cards = soup.find_all('div', class_='companyCardWrapper')

job_list = []

for card in company_cards:
    action_counts = card.find_all('span', class_='companyCardWrapper__ActionCount')
    if action_counts:
        job = action_counts[3].text.strip()  # First item is Salary
        job_list.append(job)

# Convert to Series
job_series = pd.Series(job_list)
print(job_series)


In [ ]:
# to fetch the salaries for the companies and then converting them into a series
salaries_list=[]
salaries_values=soup.find_all('span',class_='companyCardWrapper__ActionCount')
for  i in salaries_values:
    print(i.text)

In [ ]:
soup.find_all('p')

In [ ]:
pd.read_csv('Ambition_box_data.csv')[['Company','critically_rated_for']]

In [153]:
df2=pd.read_csv('Final_ambition_data.csv')
df2 = df2.drop(columns=['Unnamed: 0'])

In [154]:
df2.to_csv('Final_ambition_data.csv',index=False)

In [158]:
df2=pd.read_csv('Final_ambition_data.csv')
df2

,Company,Rating,Reviews,company_type,salary,Interview_count,Jobs_available,highly_rated_for,critically_rated_for
0,TCS,3.6k,97.1k,"IT Services & Consulting - Bangalore , Bengalu...",9.1L,11k,178,"Job Security, Work Life Balance","Promotions / Appraisal, Salary & Benefits, Wor..."
1,Accenture,3.8k,62.6k,"IT Services & Consulting - Bangalore , Bengalu...",6.1L,8.6k,11.6k,NaN,"Promotions / Appraisal, Salary & Benefits"
2,Wipro,3.7k,56.8k,"IT Services & Consulting - Bangalore , Bengalu...",4.6L,6k,6.5k,Job Security,"Promotions / Appraisal, Salary & Benefits, Wor..."
3,Cognizant,3.7k,54k,"IT Services & Consulting - Hyderabad , Secunde...",5.9L,5.9k,659,NaN,"Promotions / Appraisal, Salary & Benefits, Wor..."
4,Capgemini,3.7k,45.6k,"IT Services & Consulting - Bangalore , Bengalu...",4.5L,5k,1.6k,"Work Life Balance, Job Security","Promotions / Appraisal, Salary & Benefits, Wor..."
...,...,...,...,...,...,...,...,...,...
195,RBL Bank,3.6k,3.3k,Banking - Mumbai +253 other locations,17.1k,2.2k,32,NaN,"Promotions / Appraisal, Company Culture, Work ..."
196,Alembic Pharmaceuticals,4.0k,3.3k,Pharma - Vadodara +171 other locations,14.3k,745,70,"Salary & Benefits, Work Life Balance",NaN
197,Pidilite Industries,4.3k,3.3k,Chemicals - Mumbai +280 other locations,10.2k,1.7k,1,"Job Security, Company Culture, Work Life Balance",NaN
198,Samvardhana Motherson Group,3.7k,3.3k,Auto Components - Noida +86 other locations,12.1k,2.4k,16,Job Security,"Company Culture, Work Life Balance, Work Satis..."
